<a href="https://colab.research.google.com/github/badrinarayanan02/deep_learning/blob/main/2348507_DLLab4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Deep Neural Network Model to Predict Customer Churn Rates**

Tasked with developing a deep neural network (DNN) model to predict customer churn for a telecommunications company. The dataset provided contains various features such as customer demographics, usage patterns, and service subscription details. Objective is to implement dropout, layer-wise dropout, and Monte Carlo dropout techniques in the DNN architecture to assess their impact on model performance and generalization.